In [1]:
!pip install -q bert-score
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.7 MB/s eta 0:00:00


In [6]:

import pandas as pd
from bert_score import score
from sentence_transformers import SentenceTransformer, util

df = pd.read_csv("/kaggle/input/responses/data.csv", encoding='ISO-8859-1')


#Accuracy Evaluation using BERTScore (AI vs Ground Truth)
P = df["Ai_response "].astype(str).tolist()  # Predictions from AI
R = df["output"].astype(str).tolist()       # Ground truth doctor responses


# Compute BERTScore
_, _, F1 = score(P, R, lang="en", rescale_with_baseline=True)
df["Accuracy_F1"] = F1.tolist()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
print(df.columns.tolist())

['instruction', 'input', 'output', 'Ai_response ']


In [7]:


# Relevance Evaluation using Sentence-BERT (AI vs Query)
model = SentenceTransformer('all-MiniLM-L6-v2')

queries = df["input"].astype(str).tolist()
responses = df["Ai_response "].astype(str).tolist()

query_embeddings = model.encode(queries, convert_to_tensor=True)
response_embeddings = model.encode(responses, convert_to_tensor=True)

cos_scores = util.cos_sim(query_embeddings, response_embeddings).diagonal()
df["Relevance_Score"] = cos_scores.cpu().numpy()

#Display Results
df.to_csv("/kaggle/working/evaluated_results.csv", index=False)
df[["input", "Ai_response ", "output", "Accuracy_F1", "Relevance_Score"]].head(10)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,input,Ai_response,output,Accuracy_F1,Relevance_Score
0,"Doc,i m 36 yrs old female.I hav been suffering...",It sounds like you're dealing with a case of c...,"Hello dear, The symptoms as mentioned in your ...",-0.059371,0.751877
1,i been having real bad panic attacks and depre...,I'm really sorry you're feeling this wayit so...,Degree understand your concerns went through y...,0.015921,0.678249
2,uper back and chest light constant painDear Do...,I understand how worrying constant pain can be...,Thanks for your question on Chat Doctor. I can...,0.043250,0.659386
3,"hi, a 3 yr old son have a swelling in his righ...",Thank you for your concernits completely und...,"Thanks for your query, I have gone through you...",0.126917,0.622023
4,"I m a50 year old man, recently had a couple of...",Its possible youre dealing with recurring sc...,"Hello and welcome to Chat Doctor, Multiple lum...",0.008063,0.720624
5,i had suffered from leprosy in my childhood. a...,Ulnar neuropathy due to past leprosy can lead ...,HiT hanks for posting your query. I appreciate...,0.164659,0.711885
6,"Hello, I am 12 years of age and in my right ha...","Thank you for sharing this, and Im really sor...","Hi Dear, Welcome to Chat Doctor. Understanding...",0.021409,0.715225
7,"hi i am 21 year, about 1 week ago my right te...",I'm sorry to hear about your injuryits good ...,"HelloThanks for your query, based on the facts...",0.100170,0.444022
8,"hello sir , My name is Hasan Sheikh .My n...","Thank you for your message, Hasan. Albumin+++ ...",Hi and welcome to Chat Doctor. Thank you for y...,0.111446,0.547044
9,"Hi, may I answer your health queries right now...",I'm so sorry to hear about what you're going t...,"Hi, dairy have gone through your question. I c...",0.050178,0.399379


In [ ]:
#Trying evaluation using another LLM

In [1]:
!pip install -q transformers accelerate

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
import torch

# Load your CSV
df = pd.read_csv("/kaggle/input/responses/data.csv", encoding='ISO-8859-1')
df.columns = df.columns.str.strip()



In [4]:
from huggingface_hub import notebook_login

# Login to Hugging Face (only needed once)
notebook_login()

In [11]:

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)


generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:1


In [ ]:


# Function to prompt Mistral to evaluate accuracy
def evaluate_accuracy_with_mistral(question, ai_response, reference):
    prompt = (
        f"Evaluate the accuracy of the following AI-generated response compared to the reference medical answer.\n\n"
        f"Medical Question: {question}\n\n"
        f"AI Response: {ai_response}\n\n"
        f"Reference Answer: {reference}\n\n"
        f"Is the AI response factually accurate compared to the reference? Answer 'Yes' or 'No' and briefly explain why."
    )
    result = generator(prompt, max_new_tokens=100, do_sample=False)
    return result[0]["generated_text"]

# Evaluate and store results
mistral_evaluations = []
for _, row in df.iterrows():
    evaluation = evaluate_accuracy_with_mistral(row["input"], row["Ai_response"], row["output"])
    mistral_evaluations.append(evaluation)

df["Mistral_Accuracy_Eval"] = mistral_evaluations

import ace_tools as tools; tools.display_dataframe_to_user(name="Mistral Accuracy Evaluation", dataframe=df)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
df[["input", "Ai_response ", "output", "Mistral_Accuracy_Eval"]].head(10)